## Proyecto Curso I

##### Acerca del dataset utilizado

URL Dataset: https://www.kaggle.com/c/bike-sharing-demand/

**You must predict the total count of bikes rented during each hour covered by the test set**, using only information available prior to the rental period.

**Data Fields:**
- datetime - hourly date + timestamp  
- season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
- holiday - whether the day is considered a holiday
- workingday - whether the day is neither a weekend nor holiday
- weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- temp - temperature in Celsius
- atemp - "feels like" temperature in Celsius
- humidity - relative humidity
- windspeed - wind speed
- casual - number of non-registered user rentals initiated
- registered - number of registered user rentals initiated
- count - number of total rentals

#### Inspección del dataset

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../data/bike_rental_dataset.csv")
df.head() # count es el targe

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [ ]:
df["datetime"] = pd.to_datetime(df["datetime"])
df["hour"] = df["datetime"].dt.hour
df["month"] = df["datetime"].dt.month
df["dayofweek"] = df["datetime"].dt.dayofweek

In [35]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,month,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0,1,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2,1,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,3,1,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,4,1,5


In [36]:
df.corr()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,month,dayofweek
datetime,1.000000,0.480021,0.010988,-0.003658,-0.005048,0.180986,0.181823,0.032856,-0.086888,0.172728,0.314879,0.310187,-0.005663,0.494087,-0.004676
season,0.480021,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.096758,0.164011,0.163439,-0.006546,0.971524,-0.010553
holiday,0.010988,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,0.043799,-0.020956,-0.005393,-0.000354,0.001731,-0.191832
workingday,-0.003658,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,-0.319111,0.119460,0.011594,0.002780,-0.003394,-0.704267
weather,-0.005048,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.135918,-0.109340,-0.128655,-0.022740,0.012144,-0.047692
temp,0.180986,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.467097,0.318571,0.394454,0.145430,0.257589,-0.038466
atemp,0.181823,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.462067,0.314635,0.389784,0.140343,0.264173,-0.040235
humidity,0.032856,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.348187,-0.265458,-0.317371,-0.278011,0.204537,-0.026507
windspeed,-0.086888,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,0.092276,0.091052,0.101369,0.146631,-0.150192,-0.024804
casual,0.172728,0.096758,0.043799,-0.319111,-0.135918,0.467097,0.462067,-0.348187,0.092276,1.000000,0.497250,0.690414,0.302045,0.092722,0.246959


In [38]:
## temp & humidity estan muy correlacionadas --> quedarse solo con 1
## casual & registered tienen una correlación 0.5 --> quedarse solo con 1

X = df.drop(columns=["registered", "casual", "count", "datetime"], axis= 1)
y = df["count"]
X.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour,month,dayofweek
0,1,0,0,1,9.84,14.395,81,0.0,0,1,5
1,1,0,0,1,9.02,13.635,80,0.0,1,1,5
2,1,0,0,1,9.02,13.635,80,0.0,2,1,5
3,1,0,0,1,9.84,14.395,75,0.0,3,1,5
4,1,0,0,1,9.84,14.395,75,0.0,4,1,5


#### Definición de clases y métodos para el proyecto a utilizar

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np


class MachineLearningProcessor:
    def __init__(self, name):
        self.name = name
        self.data = pd.read_csv("../data/bike_rental_dataset.csv")
        self.pre_process_data()

    def pre_process_data(self):
        self.data["datetime"] = pd.to_datetime(self.data["datetime"])
        self.data["hour"] = self.data["datetime"].dt.hour
        self.data["month"] = self.data["datetime"].dt.month
        self.data["dayofweek"] = self.data["datetime"].dt.dayofweek
        
        self.X = self.data.drop(columns=["registered", "casual", "count", "datetime"], axis= 1)
        self.y = self.data["count"]

    def split_data(self):
        """Se utiliza para entrenar el modelo"""
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, random_state = 100, test_size = 0.2)

    def train(self, model):
        """Se utiliza para entrenar el modelo elegido con el método fit"""
        print(f"1. Iniciando el split de datos...")
        self.split_data()
        print(f"2. Iniciando el entrenamiento del modelo...")
        #self.fitted_model = model.fit(self.X_train, self.y_train)
        self.y_train_log = np.log1p(self.y_train)
        self.fitted_model_log = model.fit(self.X_train, self.y_train_log)
        print(f"3. Entrenamiento finalizado")
        ## Agregando el entrenamiento en log (porque tengo predicciones negativas y es la métrica del dataset)

    def predictions(self):
        """Se realizan las predicciones del modelo entrenado"""
        #self.predictions = self.fitted_model.predict(self.X_test)
        y_pred_log = self.fitted_model_log.predict(self.X_test)
        self.predictions_log = np.expm1(y_pred_log)
        self.predictions_log = np.maximum(0, self.predictions_log)

    def metrics(self):
        """Se construyen e imprimen las métricas principales del modelo de regresión"""
        mse = mean_squared_error(self.y_test, self.predictions_log)
        mae = mean_absolute_error(self.y_test, self.predictions_log)
        r2 = r2_score(self.y_test, self.predictions_log)
        rmse = np.sqrt(mse)
        rmsle = np.sqrt(mean_squared_log_error(self.y_test, self.predictions_log))
        
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")
        print(f"RMSLE: {rmsle}")
        print(f"r2_score: {r2}")


In [43]:
regression_model = MachineLearningProcessor(name= "Regresion Lineal")
regression_model.train(model= LinearRegression())
regression_model.predictions()
#print(np.unique(regression_model.y_test))
#print(np.unique(regression_model.predictions))
regression_model.metrics()

## Debería usarse otro modelo para este dataset

1. Iniciando el split de datos...
2. Iniciando el entrenamiento del modelo...
3. Entrenamiento finalizado
MSE: 25372.072882716457
RMSE: 159.28613524948258
MAE: 106.01038545282573
RMSLE: 1.0260452340325403
r2_score: 0.18743645066166592
